In [1]:
#pip install pandas numpy==1.23.5 tensorflow-gpu==2.10.1 scikit-learn biopython tqdm pyfaidx

In [2]:
#conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from sklearn.metrics import confusion_matrix
import json
from metrics import compute_metrics
from data_preparation import prepare_csv_datasets
from tqdm import tqdm
import time

# --- CẤU HÌNH BENCHMARK ---
MODEL_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceAI\pretrained_model"
PREPARED_FOLDER = "prepared_data/"
RESULTS_DIR = "results/"
BATCH_SIZE = 128
CONTEXT = 5000 # Phải khớp với context lúc prepare data
test_files = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']

In [2]:
import tensorflow as tf
print("Phiên bản TF:", tf.__version__)
print("Danh sách thiết bị:", tf.config.list_physical_devices())
print("Số lượng GPU:", len(tf.config.list_physical_devices('GPU')))

Phiên bản TF: 2.10.1
Danh sách thiết bị: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Số lượng GPU: 1


### Data preparation

In [5]:
os.makedirs(PREPARED_FOLDER, exist_ok=True)
prepare_csv_datasets(test_files)

[00:57:43] Loading Genome with pyfaidx...
Sorting test_1_1_1.csv for sequential disk access...
🚀 Processing: test_1_1_1.csv (26310 rows)


Extracting: 100%|██████████| 26310/26310 [00:01<00:00, 24702.66it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | AA[GG]TT                  | ❌ (Target: GT)
Donor      | CA[GG]TA                  | ❌ (Target: GT)
Donor      | CA[AG]TA                  | ❌ (Target: GT)
Donor      | AA[GG]TA                  | ❌ (Target: GT)
Donor      | AT[GG]TG                  | ❌ (Target: GT)
Acceptor   | AT[AG]GT                  | ✅ (Target: AG)
Acceptor   | CC[AG]AG                  | ✅ (Target: AG)
Acceptor   | GA[AG]GT                  | ✅ (Target: AG)
Acceptor   | GC[AG]GT                  | ✅ (Target: AG)
Acceptor   | AG[AG]CC                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_1_1_1.csv | Speed: 8493.61 seq/s

Sorting test_2_1_1.csv for sequential disk access...
🚀 Processing: test_2_1_1.csv (35132 rows)


Extracting: 100%|██████████| 35132/35132 [00:01<00:00, 24192.44it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TA]AT                  | ❌ (Target: GT)
Donor      | GA[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TA]AA                  | ❌ (Target: GT)
Donor      | AA[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TA]AG                  | ❌ (Target: GT)
Acceptor   | CC[AG]TG                  | ✅ (Target: AG)
Acceptor   | GC[AG]CC                  | ✅ (Target: AG)
Acceptor   | TC[AG]AA                  | ✅ (Target: AG)
Acceptor   | TC[AG]GC                  | ✅ (Target: AG)
Acceptor   | TT[AG]GA                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_2_1_1.csv | Speed: 8355.16 seq/s

Sorting test_4_1_1.csv for sequential disk access...
🚀 Processing: test_4_1_1.csv (52776 rows)


...


Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TA]AG                  | ❌ (Target: GT)
Donor      | CC[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TC]AG                  | ❌ (Target: GT)
Donor      | GG[TG]AG                  | ❌ (Target: GT)
Donor      | GA[GG]TA                  | ❌ (Target: GT)
Acceptor   | TC[AG]GT                  | ✅ (Target: AG)
Acceptor   | AG[AG]AA                  | ✅ (Target: AG)
Acceptor   | AC[AG]TT                  | ✅ (Target: AG)
Acceptor   | GC[AG]GA                  | ✅ (Target: AG)
Acceptor   | AG[GT]GG                  | ❌ (Target: AG)
✅ Saved to prepared_data/test_4_1_1.csv | Speed: 8139.52 seq/s

Sorting test_10_1_1.csv for sequential disk access...
🚀 Processing: test_10_1_1.csv (105708 rows)


Extracting: 100%|██████████| 105708/105708 [00:04<00:00, 24718.31it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TT]AG                  | ❌ (Target: GT)
Donor      | CA[GG]TA                  | ❌ (Target: GT)
Donor      | AC[GG]TG                  | ❌ (Target: GT)
Donor      | GG[TC]AG                  | ❌ (Target: GT)
Donor      | AA[GG]TA                  | ❌ (Target: GT)
Acceptor   | AG[CT]GC                  | ❌ (Target: AG)
Acceptor   | GC[AG]TA                  | ✅ (Target: AG)
Acceptor   | AG[GT]GG                  | ❌ (Target: AG)
Acceptor   | AG[AG]GG                  | ✅ (Target: AG)
Acceptor   | CC[AG]CA                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_10_1_1.csv | Speed: 8423.27 seq/s

Sorting test_data.csv for sequential disk access...
🚀 Processing: test_data.csv (938297 rows)


Extracting: 100%|██████████| 938297/938297 [00:37<00:00, 25041.36it/s]



Type       | Window around center (-2 to +2) | Found?
-----------------------------------------------------------------
Donor      | GG[TG]AG                  | ❌ (Target: GT)
Donor      | GG[TG]AG                  | ❌ (Target: GT)
Donor      | GG[TA]AG                  | ❌ (Target: GT)
Donor      | GA[GG]TG                  | ❌ (Target: GT)
Donor      | AA[GG]TG                  | ❌ (Target: GT)
Acceptor   | AG[CT]GT                  | ❌ (Target: AG)
Acceptor   | CT[AG]GC                  | ✅ (Target: AG)
Acceptor   | CT[AG]TG                  | ✅ (Target: AG)
Acceptor   | AG[GG]CG                  | ❌ (Target: AG)
Acceptor   | CC[AG]GG                  | ✅ (Target: AG)
✅ Saved to prepared_data/test_data.csv | Speed: 8451.58 seq/s



In [12]:
# df = pd.read_csv(r'D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceAI\prepared_data\test_1_1_1.csv')
# # Kiểm tra 5 mẫu Donor đầu tiên
# donor_samples = df[df['Splicing_types'] == 1]['sequence'].values[:5]
# for i, seq in enumerate(donor_samples):
#     # Lấy vùng trung tâm (index 5000 là nucleotide tại vị trí pos)
#     center = seq[5000:5002] 
#     print(f"Donor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là GT)")

# # Kiểm tra 5 mẫu Acceptor đầu tiên
# acceptor_samples = df[df['Splicing_types'] == 2]['sequence'].values[:5]
# for i, seq in enumerate(acceptor_samples):
#     center = seq[5000:5002]
#     print(f"Acceptor {i}: Vị trí trung tâm là '{center}' (Kỳ vọng thường là AG)")

# def diagnose_splice_sites(df, sample_size=5):
#     print(f"{'Type':<10} | {'Window (-5 to +5 around center)':<20} | {'Found?'}")
#     print("-" * 50)
    
#     for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
#         samples = df[df['Splicing_types'] == label].sample(min(sample_size, len(df)))
#         for _, row in samples.iterrows():
#             seq = row['sequence']
#             # Lấy 11 nucleotide (vị trí 5000 là trung tâm)
#             window = seq[4995:5006]
            
#             target = "GT" if label == 1 else "AG"
#             found = "✅" if target in window else "❌"
            
#             # Highlight vị trí trung tâm bằng dấu ngoặc []
#             display_win = window[:5] + "[" + window[5:7] + "]" + window[7:]
#             print(f"{name:<10} | {display_win:<20} | {found} (Target: {target})")

# # Chạy sau khi load df từ prepared_data
# diagnose_splice_sites(df)

### Inference

In [3]:
# Load tất cả models 1 lần duy nhất bên ngoài vòng lặp file
print("Loading Ensemble Models...")
models = []
for i in range(1, 6):
    model_path = os.path.join(MODEL_FOLDER, f"spliceai{i}.h5")
    models.append(tf.keras.models.load_model(model_path, compile=False))
print(f"Successfully loaded {len(models)} models.")

Loading Ensemble Models...
Successfully loaded 5 models.


In [21]:
os.makedirs(RESULTS_DIR, exist_ok=True)

def one_hot_encode(seq):
    # Vectorized Encoding (CPU-side)
    seq_code = np.zeros((len(seq), 4), dtype=np.float32)
    seq = seq.upper()
    sn = np.frombuffer(seq.encode('ascii'), dtype=np.int8)
    seq_code[sn == ord('A'), 0] = 1.0
    seq_code[sn == ord('C'), 1] = 1.0
    seq_code[sn == ord('G'), 2] = 1.0
    seq_code[sn == ord('T'), 3] = 1.0
    return seq_code

def create_optimized_dataset(sequences, batch_size):
    def gen():
        for s in sequences:
            yield one_hot_encode(s)
            
    ds = tf.data.Dataset.from_generator(
        gen,
        output_signature=tf.TensorSpec(shape=(2*CONTEXT+1, 4), dtype=tf.float32)
    )
    # Chỉ cache nếu số lượng mẫu nhỏ (ví dụ < 50,000)
    if len(sequences) < 50000:
        return ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

def debug_one_hot_encoding(sequences, sample_size=1):
    """
    Kiểm tra xem One-hot encoding có khớp với chuỗi gốc hay không.
    """
    # Lấy một vài mẫu ngẫu nhiên
    indices = np.random.choice(len(sequences), sample_size)
    map_dict = {0: 'A', 1: 'C', 2: 'G', 3: 'T'}
    
    print("--- ĐANG KIỂM TRA ONE-HOT ENCODING ---")
    
    for idx in indices:
        original_seq = sequences[idx]
        encoded_seq = one_hot_encode(original_seq)
        
        # 1. Kiểm tra Shape
        expected_shape = (2 * CONTEXT + 1, 4)
        if encoded_seq.shape != expected_shape:
            print(f"❌ LỖI SHAPE: Kỳ vọng {expected_shape}, nhận được {encoded_seq.shape}")
            return
            
        # 2. Giải mã ngược (Decode)
        # Tìm vị trí index có giá trị 1.0, nếu toàn 0 thì trả về -1 (đại diện cho 'N')
        decoded_indices = np.where(encoded_seq == 1.0)[1]
        # Xử lý trường hợp hàng toàn 0 (ký tự 'N' hoặc lỗi)
        decoded_seq_list = []
        for row in encoded_seq:
            if np.sum(row) == 0:
                decoded_seq_list.append('N')
            else:
                decoded_seq_list.append(map_dict[np.argmax(row)])
        
        decoded_seq = "".join(decoded_seq_list)
        
        # 3. So sánh chuỗi gốc và chuỗi giải mã (chỉ kiểm tra các ký tự ACGT)
        # Lưu ý: Biopython/pyfaidx có thể trả về chuỗi hỗn hợp, ta nên .upper()
        if original_seq.upper() == decoded_seq:
            print(f"✅ HÀNG {idx}: Khớp hoàn toàn (Độ dài: {len(decoded_seq)})")
        else:
            print(f"⚠️ HÀNG {idx}: CÓ SỰ KHÁC BIỆT!")
            # In ra 10 ký tự đầu tiên để so sánh
            print(f"   Gốc: {original_seq[:10]}...")
            print(f"   Mã : {decoded_seq[:10]}...")

        # 4. Kiểm tra xem có vector nào bị lỗi (ví dụ có 2 số 1 trong 1 hàng)
        invalid_rows = np.where(np.sum(encoded_seq, axis=1) > 1)[0]
        if len(invalid_rows) > 0:
            print(f"❌ LỖI LOGIC: Có {len(invalid_rows)} hàng chứa nhiều hơn một giá trị 1.0")


pbar_files = tqdm(test_files, desc="Overall Progress", unit="file")

for csv_file in pbar_files:
    path = os.path.join(PREPARED_FOLDER, csv_file)
    if not os.path.exists(path): continue
    
    pbar_files.set_postfix({"file": csv_file})
    
    df = pd.read_csv(path)
    debug_one_hot_encoding(df['sequence'].values, sample_size=3)
    y_true = df['Splicing_types'].values
    ds = create_optimized_dataset(df['sequence'], BATCH_SIZE)

    total_probs = None 
    encoding_time = 0
    inference_start = time.time()

    for i, model in enumerate(tqdm(models, desc=f" ↪ Models Ensemble", leave=False)):
        # Đo thời gian encoding ở model đầu tiên (lúc generator thực sự chạy)
        if i == 0:
            enc_start = time.time()
            preds = model.predict(ds, verbose=1)
            encoding_time = time.time() - enc_start
        else:
            preds = model.predict(ds, verbose=1)
        
        # --- SỬA LỖI TRUY CẬP CHỈ SỐ (IndexError Fix) ---
        if preds.ndim == 3:
            if preds.shape[1] > CONTEXT:
                # Trường hợp model trả về cả chuỗi (vd: 10001)
                center_probs = preds[:, CONTEXT, :]
            else:
                # Trường hợp model trả về singleton dimension (vd: shape (batch, 1, 3))
                center_probs = preds[:, 0, :]
        else:
            # Trường hợp model trả về 2D (batch, classes)
            center_probs = preds
        
        if total_probs is None:
            total_probs = center_probs
        else:
            total_probs += center_probs
            
    avg_probs = total_probs / len(models)
    total_duration = time.time() - inference_start
    
    # Mapping nhãn theo đúng thứ tự (None, Donor, Acceptor)
    probs_mapped = np.stack([avg_probs[:, 0], avg_probs[:, 2], avg_probs[:, 1]], axis=1)
    # preds_mapped = np.argmax(probs_mapped, axis=1)

    THRESHOLD = 0.0000025 # Ngưỡng nhạy bén cho Donor/Acceptor

    # Lấy xác suất Donor và Acceptor
    p_donor = probs_mapped[:, 1]
    p_acceptor = probs_mapped[:, 2]

    # Tạo nhãn dự đoán: Ưu tiên Donor/Acceptor nếu vượt ngưỡng
    preds_mapped = np.zeros(len(probs_mapped), dtype=int)
    for i in range(len(probs_mapped)):
        if p_donor[i] > THRESHOLD or p_acceptor[i] > THRESHOLD:
            preds_mapped[i] = 1 if p_donor[i] > p_acceptor[i] else 2
        else:
            preds_mapped[i] = 0
    
    print(f"\n--- Kiểm tra phân phối xác suất cho {csv_file} ---")
    for i in [1, 2]:
        mask = (y_true == i)
        if np.any(mask):
            probs_label = probs_mapped[mask, i]
            mean_p = np.mean(probs_label)
            max_p = np.max(probs_label)
            
            # Lấy Top 10 giá trị thấp nhất (để xem model tự tin thấp đến mức nào)
            num_elements = len(probs_label)
            k = min(10, num_elements) 
            
            # np.partition đưa k phần tử nhỏ nhất lên đầu
            top_k_min = np.sort(np.partition(probs_label, k-1)[:k])
            
            top_str = ", ".join([f"{val:.10f}" for val in top_k_min])
            print(f"   Nhãn {i}: Avg = {mean_p:.6f}, Max = {max_p:.6f}, Top {k} Min = [{top_str}]")
        else:
            print(f"   Nhãn {i}: Không có dữ liệu trong y_true")

    try:
        results = compute_metrics(y_true, preds_mapped, probs=probs_mapped, k=2)
        
        # Lưu thông tin benchmarking
        results['benchmarking'] = {
            'encoding_time_sec': round(encoding_time, 4),
            'total_inference_time_sec': round(total_duration, 4),
            'samples_per_second': round(len(df) * len(models) / total_duration, 2)
        }
        
        cm = confusion_matrix(y_true, preds_mapped)
        output = {"metrics": results, "confusion_matrix": cm.tolist()}
        
        json_path = os.path.join(RESULTS_DIR, csv_file.replace('.csv', '_results.json'))
        with open(json_path, 'w') as f:
            json.dump(output, f, indent=4)
            
        tqdm.write(f"✅ {csv_file} | Enc: {encoding_time:.2f}s | Total: {total_duration:.2f}s")
            
    except Exception as e:
        print(f"\n❌ Error with {csv_file}: {e}")

print("\n✨ ALL DATASETS PROCESSED SUCCESSFULLY!")

Overall Progress:   0%|          | 0/5 [00:00<?, ?file/s, file=test_1_1_1.csv]

--- ĐANG KIỂM TRA ONE-HOT ENCODING ---
✅ HÀNG 10342: Khớp hoàn toàn (Độ dài: 10001)
✅ HÀNG 3732: Khớp hoàn toàn (Độ dài: 10001)
✅ HÀNG 4845: Khớp hoàn toàn (Độ dài: 10001)


206/206 [==============================] - 63s 306ms/step


206/206 [==============================] - 62s 302ms/step


206/206 [==============================] - 62s 301ms/step


...

   Nhãn 1: Avg = 0.382683, Max = 0.999861, Top 10 Min = [0.0000000002, 0.0000000006, 0.0000000006, 0.0000000007, 0.0000000010, 0.0000000013, 0.0000000017, 0.0000000017, 0.0000000017, 0.0000000018]
   Nhãn 2: Avg = 0.322296, Max = 0.999870, Top 10 Min = [0.0000000009, 0.0000000009, 0.0000000015, 0.0000000019, 0.0000000020, 0.0000000023, 0.0000000028, 0.0000000029, 0.0000000033, 0.0000000038]
✅ test_4_1_1.csv | Enc: 124.14s | Total: 619.71s
--- ĐANG KIỂM TRA ONE-HOT ENCODING ---
✅ HÀNG 1809: Khớp hoàn toàn (Độ dài: 10001)
✅ HÀNG 978: Khớp hoàn toàn (Độ dài: 10001)
✅ HÀNG 2269: Khớp hoàn toàn (Độ dài: 10001)


826/826 [==============================] - 248s 301ms/step


    753/Unknown - 226s 300ms/step

Overall Progress:  60%|██████    | 3/5 [30:29<20:19, 609.89s/file, file=test_10_1_1.csv]


KeyboardInterrupt: 